In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import caffe

In [18]:
import os
from os.path import join
import torch
import sys
import numpy as np
from pathlib import Path
import sys
sys.path.append(r"/n/home12/binxuwang/Github/pytorch-caffe")

#%%
# Depend on 2 packages, you should clone from
# https://github.com/Animadversio/pytorch-receptive-field
# https://github.com/Animadversio/pytorch-caffe.git
# from sys import platform
# if platform == "linux":  # CHPC cluster
#     homedir = Path(os.path.expanduser('~'))
#     netsdir = os.path.join(homedir, 'Generate_DB/nets')
#     sys.path.append(join(homedir,"pytorch-caffe"))
#     sys.path.append(join(homedir,"pytorch-receptive-field"))
#     sys.path.append(join(homedir,"PerceptualSimilarity"))  # should be added there!)
#     # ckpt_path = {"vgg16": "/scratch/binxu/torch/vgg16-397923af.pth"}
# else:
#     if os.environ['COMPUTERNAME'] == 'DESKTOP-9DDE2RH':  # PonceLab-Desktop 3
#         sys.path.append(r"D:\Github\pytorch-caffe")
#         sys.path.append(r"D:\Github\pytorch-receptive-field")
#         sys.path.append(r"D:\Github\PerceptualSimilarity")
#         homedir = "D:/Generator_DB_Windows"
#         netsdir = os.path.join(homedir, 'nets')
#     elif os.environ['COMPUTERNAME'] == 'PONCELAB-ML2C':  # PonceLab-Desktop Victoria
#         sys.path.append(r"C:\Users\ponce\Documents\GitHub\pytorch-caffe")
#         sys.path.append(r"C:\Users\ponce\Documents\GitHub\pytorch-receptive-field")
#         homedir = r"C:\Users\ponce\Documents\Generator_DB_Windows"
#         netsdir = os.path.join(homedir, 'nets')
#     elif os.environ['COMPUTERNAME'] == 'DESKTOP-MENSD6S':  # Home_WorkStation
#         sys.path.append(r"E:\Github_Projects\pytorch-caffe")
#         sys.path.append(r"E:\Github_Projects\pytorch-receptive-field")
#         sys.path.append(r"E:\Github_Projects\PerceptualSimilarity")
#         homedir = "E:/Monkey_Data/Generator_DB_Windows"
#         netsdir = os.path.join(homedir, 'nets')
#     elif os.environ['COMPUTERNAME'] == 'DESKTOP-9LH02U9':  # Home_WorkStation Victoria
#         sys.path.append(r"C:\Users\zhanq\OneDrive - Washington University in St. Louis\GitHub\pytorch-caffe")
#         sys.path.append(r"C:\Users\zhanq\OneDrive - Washington University in St. Louis\GitHub\pytorch-receptive-field")
#         homedir = "C:/Users/zhanq/OneDrive - Washington University in St. Louis/Generator_DB_Windows"
#         netsdir = os.path.join(homedir, 'nets')
#     else:
#         sys.path.append("D:\Github\pytorch-caffe")
#         homedir = os.path.expanduser('~')
#         netsdir = os.path.join(homedir, 'Documents/nets')
netsdir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Projects/ReprInvertNet/DeePSim_orig/" # release_deepsim_v0

def GAN_path(name):
    if name == "fc6":
        save_path = os.path.join(netsdir, r"upconv/fc6/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/fc6/generator.prototxt") 
        weightfile = os.path.join(netsdir, r'upconv/fc6/generator.caffemodel')
    elif name == "fc7":
        save_path = os.path.join(netsdir, r"upconv/fc7/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/fc7/generator.prototxt")  
        weightfile = os.path.join(netsdir, r'upconv/fc7/generator.caffemodel')  
    elif name == "fc8":
        save_path = os.path.join(netsdir, r"upconv/fc8/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/fc8/generator.prototxt") 
        weightfile = os.path.join(netsdir, r'upconv/fc8/generator.caffemodel')
    elif name == "pool5":
        save_path = os.path.join(netsdir, r"upconv/pool5/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/pool5/generator.prototxt") 
        weightfile = os.path.join(netsdir, r'upconv/pool5/generator.caffemodel')
    elif name == "conv4":
        save_path = os.path.join(netsdir, r"upconv/conv4/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/conv4/generator.prototxt") 
        weightfile = os.path.join(netsdir, r'upconv/conv4/generator.caffemodel')
    elif name == "conv3":
        save_path = os.path.join(netsdir, r"upconv/conv3/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/conv3/generator.prototxt") 
        weightfile = os.path.join(netsdir, r'upconv/conv3/generator.caffemodel')
    elif name == "norm2":
        save_path = os.path.join(netsdir, r"upconv/norm2/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/norm2/generator.prototxt")  
        weightfile = os.path.join(netsdir, r'upconv/norm2/generator.caffemodel')  
    elif name == "norm1":
        save_path = os.path.join(netsdir, r"upconv/norm1/generator_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/norm1/generator.prototxt")  
        weightfile = os.path.join(netsdir, r'upconv/norm1/generator.caffemodel')  
    elif name == "caffenet":
        save_path = os.path.join(netsdir, r"upconv/caffenet/caffenet_state_dict.pt")
        protofile = os.path.join(netsdir, r"upconv/caffenet/caffenet.prototxt")  
        weightfile = os.path.join(netsdir, r'upconv/caffenet/caffenet.caffemodel')  
    else:
        raise ValueError(name + 'not defined')
    return save_path, protofile, weightfile


#%% Prepare PyTorch version of the Caffe networks
def load_caffenet():
    from caffenet import CaffeNet  # Pytorch-caffe converter
    # protofile = join(netsdir, r"caffenet\caffenet.prototxt")  # 'resnet50/deploy.prototxt'
    # weightfile = join(netsdir, r'caffenet\bvlc_reference_caffenet.caffemodel')  # 'resnet50/resnet50.caffemodel'
    # save_path = join(netsdir, r"caffenet\caffenet_state_dict.pt")
    save_path, protofile, weightfile = GAN_path('caffenet')
    net = CaffeNet(protofile)
    print(net)
    if os.path.exists(save_path):
        net.load_state_dict(torch.load(save_path))
    else:
        net.load_weights(weightfile)
        torch.save(net.state_dict(), save_path)
    net.eval()
    net.verbose = False
    net.requires_grad_(requires_grad=False)
    for param in net.parameters():
        param.requires_grad = False
    return net


def load_generator(GAN="fc6"):
    from caffenet import CaffeNet  # Pytorch-caffe converter
    # netsdir = r"D:/Generator_DB_Windows/nets"
    save_path, protofile, weightfile = GAN_path(GAN)
    Generator = CaffeNet(protofile)
    print(Generator)
    if os.path.exists(save_path):
        Generator.load_state_dict(torch.load(save_path))
    else:
        Generator.load_weights(weightfile)
        torch.save(Generator.state_dict(), save_path)  # Generator.
    Generator.eval()
    Generator.verbose = False
    Generator.requires_grad_(requires_grad=False)
    for param in Generator.parameters():
        param.requires_grad = False
    return Generator

In [7]:
Generator = load_generator('norm1')

create Rconv6               (  96 x   27 x   27) -> ( 128 x   15 x   15)
create Rrelu6               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create deconv4              ( 128 x   15 x   15) -> ( 128 x   30 x   30)
create relu_deconv4         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create conv4_1              ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create relu_conv4_1         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create deconv3              ( 128 x   30 x   30) -> (  64 x   60 x   60)
create relu_deconv3         (  64 x   60 x   60) -> (  64 x   60 x   60)
create conv3_1              (  64 x   60 x   60) -> (  64 x   60 x   60)
create relu_conv3_1         (  64 x   60 x   60) ->

In [19]:
Generator = load_generator('caffenet')

create conv1                (   3 x  227 x  227) -> (  96 x   55 x   55)
create relu1                (  96 x   55 x   55) -> (  96 x   55 x   55)
create pool1                (  96 x   55 x   55) -> (  96 x   27 x   27)
create norm1                (  96 x   27 x   27) -> (  96 x   27 x   27)
create conv2                (  96 x   27 x   27) -> ( 256 x   27 x   27)
create relu2                ( 256 x   27 x   27) -> ( 256 x   27 x   27)
create pool2                ( 256 x   27 x   27) -> ( 256 x   13 x   13)
create norm2                ( 256 x   13 x   13) -> ( 256 x   13 x   13)
create conv3                ( 256 x   13 x   13) -> ( 384 x   13 x   13)
create relu3                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create conv4                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create relu4                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create conv5                ( 384 x   13 x   13) -> ( 256 x   13 x   13)
create relu5                ( 256 x   13 x   13) ->

In [5]:
Generator = load_generator('caffenet')

create conv1                (   3 x  227 x  227) -> (  96 x   55 x   55)
create relu1                (  96 x   55 x   55) -> (  96 x   55 x   55)
create pool1                (  96 x   55 x   55) -> (  96 x   27 x   27)
create norm1                (  96 x   27 x   27) -> (  96 x   27 x   27)
create conv2                (  96 x   27 x   27) -> ( 256 x   27 x   27)
create relu2                ( 256 x   27 x   27) -> ( 256 x   27 x   27)
create pool2                ( 256 x   27 x   27) -> ( 256 x   13 x   13)
create norm2                ( 256 x   13 x   13) -> ( 256 x   13 x   13)
create conv3                ( 256 x   13 x   13) -> ( 384 x   13 x   13)
create relu3                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create conv4                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create relu4                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create conv5                ( 384 x   13 x   13) -> ( 256 x   13 x   13)
create relu5                ( 256 x   13 x   13) ->

In [13]:
Generator = load_generator('fc6')
Generator = load_generator('fc7')
Generator = load_generator('fc8')

create defc7                (4096 x    1 x    1) -> (4096 x    1 x    1)
create relu_defc7           (4096 x    1 x    1) -> (4096 x    1 x    1)
create defc6                (4096 x    1 x    1) -> (4096 x    1 x    1)
create relu_defc6           (4096 x    1 x    1) -> (4096 x    1 x    1)
create defc5                (4096 x    1 x    1) -> (4096 x    1 x    1)
create relu_defc5           (4096 x    1 x    1) -> (4096 x    1 x    1)
create reshape              (4096 x    1 x    1) -> ( 256 x    4 x    4)
create deconv5              ( 256 x    4 x    4) -> ( 256 x    8 x    8)
create relu_deconv5         ( 256 x    8 x    8) -> ( 256 x    8 x    8)
create conv5_1              ( 256 x    8 x    8) -> ( 512 x    8 x    8)
create relu_conv5_1         ( 512 x    8 x    8) -> ( 512 x    8 x    8)
create deconv4              ( 512 x    8 x    8) -> ( 256 x   16 x   16)
create relu_deconv4         ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create conv4_1              ( 256 x   16 x   16) ->

In [12]:
Generator = load_generator('pool5')

create Rconv6               ( 256 x    6 x    6) -> ( 512 x    6 x    6)
create Rrelu6               ( 512 x    6 x    6) -> ( 512 x    6 x    6)
create Rconv7               ( 512 x    6 x    6) -> ( 512 x    6 x    6)
create Rrelu7               ( 512 x    6 x    6) -> ( 512 x    6 x    6)
create Rconv8               ( 512 x    6 x    6) -> ( 512 x    4 x    4)
create Rrelu8               ( 512 x    4 x    4) -> ( 512 x    4 x    4)
create deconv5              ( 512 x    4 x    4) -> ( 256 x    8 x    8)
create relu_deconv5         ( 256 x    8 x    8) -> ( 256 x    8 x    8)
create conv5_1              ( 256 x    8 x    8) -> ( 512 x    8 x    8)
create relu_conv5_1         ( 512 x    8 x    8) -> ( 512 x    8 x    8)
create deconv4              ( 512 x    8 x    8) -> ( 256 x   16 x   16)
create relu_deconv4         ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create conv4_1              ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create relu_conv4_1         ( 256 x   16 x   16) ->

### Build CaffeNet

In [ ]:
import torch
import torch.nn as nn
from collections import OrderedDict

In [ ]:
CNN = load_caffenet()

create conv1                (   3 x  227 x  227) -> (  96 x   55 x   55)
create relu1                (  96 x   55 x   55) -> (  96 x   55 x   55)
create pool1                (  96 x   55 x   55) -> (  96 x   27 x   27)
create norm1                (  96 x   27 x   27) -> (  96 x   27 x   27)
create conv2                (  96 x   27 x   27) -> ( 256 x   27 x   27)
create relu2                ( 256 x   27 x   27) -> ( 256 x   27 x   27)
create pool2                ( 256 x   27 x   27) -> ( 256 x   13 x   13)
create norm2                ( 256 x   13 x   13) -> ( 256 x   13 x   13)
create conv3                ( 256 x   13 x   13) -> ( 384 x   13 x   13)
create relu3                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create conv4                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create relu4                ( 384 x   13 x   13) -> ( 384 x   13 x   13)
create conv5                ( 384 x   13 x   13) -> ( 256 x   13 x   13)
create relu5                ( 256 x   13 x   13) ->

In [15]:
caffenet_th = nn.Sequential(OrderedDict([
    # Layer 1
    ('conv1', nn.Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))),
    ('relu1', nn.ReLU(inplace=True)),
    ('pool1', nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)),
    ('norm1', nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)),
    # Layer 2
    ('conv2', nn.Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)),
    ('relu2', nn.ReLU(inplace=True)),
    ('pool2', nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)),
    ('norm2', nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)),
    # Layer 3
    ('conv3', nn.Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu3', nn.ReLU(inplace=True)),
    # Layer 4
    ('conv4', nn.Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)),
    ('relu4', nn.ReLU(inplace=True)),
    # Layer 5
    ('conv5', nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)),
    ('relu5', nn.ReLU(inplace=True)),
    ('pool5', nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)),
    # Fully connected layers
    ('flatten', nn.Flatten()),
    ('fc6', nn.Linear(9216, 4096)),
    ('relu6', nn.ReLU(inplace=True)),
    ('fc7', nn.Linear(4096, 4096)),
    ('relu7', nn.ReLU(inplace=True)),
    ('fc8', nn.Linear(4096, 1000))
]))
state_dict = {}
for k, v in CNN.state_dict().items():
    if '.1.' in k:
        state_dict[k.replace('.1.', '.')] = v
    else:
        state_dict[k] = v.cpu()
caffenet_th.load_state_dict(state_dict)
latent_shape = (3, 227, 227)
latents = torch.randn(1, *latent_shape)
out = CNN(latents)
# out["generated"]
out_th = caffenet_th(latents)
assert torch.allclose(out["fc8"], out_th)
# For the Eltwise SUM operation, a custom module or function should be added outside of Sequential.

In [16]:
torch.save(caffenet_th.state_dict(), "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Projects/ReprInvertNet/DeePSim_orig/upconv/caffenet/caffenet_simple.pt")

### Build generators for norm1-4

### norm1

In [12]:
Generator = load_generator('norm1')
model_th = nn.Sequential(OrderedDict([
    ('Rconv6', nn.Conv2d(96, 128, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))),
    ('Rrelu6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv7', nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('Rrelu7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv8', nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('Rrelu8', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv4', nn.ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv4_1', nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv3', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv3_1', nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv2', nn.ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv2_1', nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv2_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv1', nn.ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('conv1_1', nn.Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('tanh', nn.Tanh())
]))
model_th.load_state_dict(Generator.state_dict())
latent_shape = (96, 27, 27)
latents = torch.randn(1, *latent_shape)
out = Generator(latents)
# out["generated"]
out_th = model_th(latents)
assert torch.allclose(out["generated"], out_th)
# For the Eltwise SUM operation, a custom module or function should be added outside of Sequential.

create Rconv6               (  96 x   27 x   27) -> ( 128 x   15 x   15)
create Rrelu6               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create deconv4              ( 128 x   15 x   15) -> ( 128 x   30 x   30)
create relu_deconv4         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create conv4_1              ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create relu_conv4_1         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create deconv3              ( 128 x   30 x   30) -> (  64 x   60 x   60)
create relu_deconv3         (  64 x   60 x   60) -> (  64 x   60 x   60)
create conv3_1              (  64 x   60 x   60) -> (  64 x   60 x   60)
create relu_conv3_1         (  64 x   60 x   60) ->

### norm2

In [16]:
Generator = load_generator('norm2')
model_th = nn.Sequential(OrderedDict([
    ('Rconv6', nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))),
    ('Rrelu6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv7', nn.Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('Rrelu7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv8', nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('Rrelu8', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv4', nn.ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv4_1', nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv3', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv3_1', nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv2', nn.ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv2_1', nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv2_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv1', nn.ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('conv1_1', nn.Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('tanh', nn.Tanh())
]))
model_th.load_state_dict(Generator.state_dict())
latent_shape = (256, 13, 13)
latents = torch.randn(1, *latent_shape)
out = Generator(latents)
# out["generated"]
out_th = model_th(latents)
assert torch.allclose(out["generated"], out_th)
# For the Eltwise SUM operation, a custom module or function should be added outside of Sequential.

create Rconv6               ( 256 x   13 x   13) -> ( 256 x   15 x   15)
create Rrelu6               ( 256 x   15 x   15) -> ( 256 x   15 x   15)
create Rconv7               ( 256 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create deconv4              ( 128 x   15 x   15) -> ( 128 x   30 x   30)
create relu_deconv4         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create conv4_1              ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create relu_conv4_1         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create deconv3              ( 128 x   30 x   30) -> (  64 x   60 x   60)
create relu_deconv3         (  64 x   60 x   60) -> (  64 x   60 x   60)
create conv3_1              (  64 x   60 x   60) -> (  64 x   60 x   60)
create relu_conv3_1         (  64 x   60 x   60) ->

### conv3

In [18]:
Generator = load_generator('conv3')
model_th = nn.Sequential(OrderedDict([
    ('Rconv6', nn.Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))),
    ('Rrelu6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv7', nn.Conv2d(384, 512, kernel_size=(3, 3), stride=(1, 1))),
    ('Rrelu7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv8', nn.Conv2d(512, 512, kernel_size=(2, 2), stride=(1, 1))),
    ('Rrelu8', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv5', nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv5', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv5_1', nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv5_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv4', nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv4_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv3', nn.ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv3_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv2', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv2_1', nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv2_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv1', nn.ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv1_1', nn.Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('tanh', nn.Tanh())
]))
model_th.load_state_dict(Generator.state_dict())
latent_shape = (384, 13, 13)
latents = torch.randn(1, *latent_shape)
out = Generator(latents)
# out["generated"]
out_th = model_th(latents)
assert torch.allclose(out["generated"], out_th)
# For the Eltwise SUM operation, a custom module or function should be added outside of Sequential.

create Rconv6               ( 384 x   13 x   13) -> ( 384 x   11 x   11)
create Rrelu6               ( 384 x   11 x   11) -> ( 384 x   11 x   11)
create Rconv7               ( 384 x   11 x   11) -> ( 512 x    9 x    9)
create Rrelu7               ( 512 x    9 x    9) -> ( 512 x    9 x    9)
create Rconv8               ( 512 x    9 x    9) -> ( 512 x    8 x    8)
create Rrelu8               ( 512 x    8 x    8) -> ( 512 x    8 x    8)
create deconv5              ( 512 x    8 x    8) -> ( 256 x   16 x   16)
create relu_deconv5         ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create conv5_1              ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create relu_conv5_1         ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create deconv4              ( 256 x   16 x   16) -> ( 128 x   32 x   32)
create relu_deconv4         ( 128 x   32 x   32) -> ( 128 x   32 x   32)
create conv4_1              ( 128 x   32 x   32) -> ( 128 x   32 x   32)
create relu_conv4_1         ( 128 x   32 x   32) ->

### conv4

In [20]:
Generator = load_generator('conv4')
model_th = nn.Sequential(OrderedDict([
    ('Rconv6', nn.Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))),
    ('Rrelu6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv7', nn.Conv2d(384, 512, kernel_size=(3, 3), stride=(1, 1))),
    ('Rrelu7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('Rconv8', nn.Conv2d(512, 512, kernel_size=(2, 2), stride=(1, 1))),
    ('Rrelu8', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv5', nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv5', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv5_1', nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv5_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv4', nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv4_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv3', nn.ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv3_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv2', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv2_1', nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('relu_conv2_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('deconv1', nn.ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
    ('relu_deconv1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
    ('conv1_1', nn.Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
    ('tanh', nn.Tanh())
]))
model_th.load_state_dict(Generator.state_dict())
latent_shape = (384, 13, 13)
latents = torch.randn(1, *latent_shape)
out = Generator(latents)
# out["generated"]
out_th = model_th(latents)
assert torch.allclose(out["generated"], out_th)
# For the Eltwise SUM operation, a custom module or function should be added outside of Sequential.

create Rconv6               ( 384 x   13 x   13) -> ( 384 x   11 x   11)
create Rrelu6               ( 384 x   11 x   11) -> ( 384 x   11 x   11)
create Rconv7               ( 384 x   11 x   11) -> ( 512 x    9 x    9)
create Rrelu7               ( 512 x    9 x    9) -> ( 512 x    9 x    9)
create Rconv8               ( 512 x    9 x    9) -> ( 512 x    8 x    8)
create Rrelu8               ( 512 x    8 x    8) -> ( 512 x    8 x    8)
create deconv5              ( 512 x    8 x    8) -> ( 256 x   16 x   16)
create relu_deconv5         ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create conv5_1              ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create relu_conv5_1         ( 256 x   16 x   16) -> ( 256 x   16 x   16)
create deconv4              ( 256 x   16 x   16) -> ( 128 x   32 x   32)
create relu_deconv4         ( 128 x   32 x   32) -> ( 128 x   32 x   32)
create conv4_1              ( 128 x   32 x   32) -> ( 128 x   32 x   32)
create relu_conv4_1         ( 128 x   32 x   32) ->

In [10]:
Generator = load_generator('norm1')
Generator = load_generator('norm2')
Generator = load_generator('conv3')
Generator = load_generator('conv4')


create Rconv6               (  96 x   27 x   27) -> ( 128 x   15 x   15)
create Rrelu6               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu7               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rconv8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create Rrelu8               ( 128 x   15 x   15) -> ( 128 x   15 x   15)
create deconv4              ( 128 x   15 x   15) -> ( 128 x   30 x   30)
create relu_deconv4         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create conv4_1              ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create relu_conv4_1         ( 128 x   30 x   30) -> ( 128 x   30 x   30)
create deconv3              ( 128 x   30 x   30) -> (  64 x   60 x   60)
create relu_deconv3         (  64 x   60 x   60) -> (  64 x   60 x   60)
create conv3_1              (  64 x   60 x   60) -> (  64 x   60 x   60)
create relu_conv3_1         (  64 x   60 x   60) ->

In [ ]:
class upconvGAN(nn.Module):
    def __init__(self, name="fc6", pretrained=True, shuffled=False):
        super(upconvGAN, self).__init__()
        self.name = name
        if name == "fc6" or name == "fc7":
            self.G = nn.Sequential(OrderedDict([
        ('defc7', nn.Linear(in_features=4096, out_features=4096, bias=True)),
        ('relu_defc7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('defc6', nn.Linear(in_features=4096, out_features=4096, bias=True)),
        ('relu_defc6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('defc5', nn.Linear(in_features=4096, out_features=4096, bias=True)),
        ('relu_defc5', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('reshape', View((-1, 256, 4, 4))),
        ('deconv5', nn.ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv5', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('conv5_1', nn.ConvTranspose2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('relu_conv5_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv4', nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('conv4_1', nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv3', nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('conv3_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv2', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv1', nn.ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv0', nn.ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
            ]))
            self.codelen = self.G[0].in_features
        elif name == "fc8":
            self.G = nn.Sequential(OrderedDict([
  ("defc7", nn.Linear(in_features=1000, out_features=4096, bias=True)),
  ("relu_defc7", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("defc6", nn.Linear(in_features=4096, out_features=4096, bias=True)),
  ("relu_defc6", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("defc5", nn.Linear(in_features=4096, out_features=4096, bias=True)),
  ("relu_defc5", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("reshape", View((-1, 256, 4, 4))),
  ("deconv5", nn.ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
  ("relu_deconv5", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("conv5_1", nn.ConvTranspose2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
  ("relu_conv5_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("deconv4", nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
  ("relu_deconv4", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("conv4_1", nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
  ("relu_conv4_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("deconv3", nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
  ("relu_deconv3", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("conv3_1", nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
  ("relu_conv3_1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("deconv2", nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
  ("relu_deconv2", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("deconv1", nn.ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
  ("relu_deconv1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
  ("deconv0", nn.ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
  ]))
            self.codelen = self.G[0].in_features
        elif name == "pool5":
            self.G = nn.Sequential(OrderedDict([
        ('Rconv6', nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('Rrelu6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('Rconv7', nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('Rrelu7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('Rconv8', nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))),
        ('Rrelu8', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv5', nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv5', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('conv5_1', nn.ConvTranspose2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('relu_conv5_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv4', nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('conv4_1', nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv3', nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('conv3_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
        ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv2', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv1', nn.ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
        ('relu_deconv1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
        ('deconv0', nn.ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))), ]))
            self.codelen = self.G[0].in_channels
            self.latent_shape = (256, 4, 4)
        elif name == "conv4":
            self.G = nn.Sequential(OrderedDict([
                ('Rconv6', nn.Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))),
                ('Rrelu6', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('Rconv7', nn.Conv2d(384, 512, kernel_size=(3, 3), stride=(1, 1))),
                ('Rrelu7', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('Rconv8', nn.Conv2d(512, 512, kernel_size=(2, 2), stride=(1, 1))),
                ('Rrelu8', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('deconv5', nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
                ('relu_deconv5', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('conv5_1', nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
                ('relu_conv5_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('deconv4', nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
                ('relu_deconv4', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('conv4_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
                ('relu_conv4_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('deconv3', nn.ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
                ('relu_deconv3', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('conv3_1', nn.ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
                ('relu_conv3_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('deconv2', nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
                ('relu_deconv2', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('conv2_1', nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
                ('relu_conv2_1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('deconv1', nn.ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))),
                ('relu_deconv1', nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ('conv1_1', nn.Conv2d(16, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))),
                ('tanh', nn.Tanh())
            ]))
            self.latent_shape = (384, 13, 13)
        # load pre-trained weight from online or local folders
        if pretrained:
            if load_urls:
                SDnew = load_statedict_from_online(name)
            else:
                savepath = {"fc6": join(netsdir, r"upconv/fc6/generator_state_dict.pt"),
                            "fc7": join(netsdir, r"upconv/fc7/generator_state_dict.pt"),
                            "fc8": join(netsdir, r"upconv/fc8/generator_state_dict.pt"),
                            "pool5": join(netsdir, r"upconv/pool5/generator_state_dict.pt")}
                SD = torch.load(savepath[name])
                SDnew = OrderedDict()
                for name, W in SD.items():  # discard this inconsistency
                    name = name.replace(".1.", ".")
                    SDnew[name] = W
            self.G.load_state_dict(SDnew)
        # if shuffled:

    def forward(self, x):
        return self.G(x)[:, [2, 1, 0], :, :]

    def visualize(self, x, scale=1.0):
        raw = self.G(x)
        return torch.clamp(raw[:, [2, 1, 0], :, :] + RGB_mean.to(raw.device), 0, 255.0) / 255.0 * scale

    def visualize_batch(self, x_arr, scale=1.0, B=42, ):
        coden = x_arr.shape[0]
        img_all = []
        csr = 0  # if really want efficiency, we should use minibatch processing.
        with torch.no_grad():
            while csr < coden:
                csr_end = min(csr + B, coden)
                imgs = self.visualize(x_arr[csr:csr_end, :].cuda(), scale).cpu()
                img_all.append(imgs)
                csr = csr_end
        img_all = torch.cat(img_all, dim=0)
        return img_all

    def render(self, x, scale=1.0, B=42):  # add batch processing to avoid memory over flow for batch too large
        coden = x.shape[0]
        img_all = []
        csr = 0  # if really want efficiency, we should use minibatch processing.
        while csr < coden:
            csr_end = min(csr + B, coden)
            with torch.no_grad():
                imgs = self.visualize(torch.from_numpy(x[csr:csr_end, :]).float().cuda(), scale).permute(2,3,1,0).cpu().numpy()
            img_all.extend([imgs[:, :, :, imgi] for imgi in range(imgs.shape[3])])
            csr = csr_end
        return img_all

    def visualize_batch_np(self, codes_all_arr, scale=1.0, B=42, verbose=False):
        coden = codes_all_arr.shape[0]
        img_all = None
        csr = 0  # if really want efficiency, we should use minibatch processing.
        with torch.no_grad():
            while csr < coden:
                csr_end = min(csr + B, coden)
                imgs = self.visualize(torch.from_numpy(codes_all_arr[csr:csr_end, :]).float().cuda(), scale).cpu()
                img_all = imgs if img_all is None else torch.cat((img_all, imgs), dim=0)
                csr = csr_end
                if verbose:
                    clear_output(wait=True)
                    progress_bar(csr_end, coden, "ploting row of page: %d of %d" % (csr_end, coden))
        return img_all

In [7]:
list(out)

['feat',
 'Rconv6',
 'Rconv7',
 'Rconv8',
 'deconv4',
 'conv4_1',
 'deconv3',
 'conv3_1',
 'deconv2',
 'conv2_1',
 'deconv1',
 'conv1_1',
 'conv1_1_tanh',
 'generated']

tensor([[[[ 1.5243e-04,  3.0583e-04,  4.4513e-06,  ...,  4.0827e-04,
           -1.3420e-04, -5.0331e-04],
          [ 3.1363e-04,  5.4964e-04,  7.0502e-04,  ..., -7.4048e-04,
           -8.3837e-04, -1.5165e-04],
          [ 1.5197e-04,  5.9897e-04,  7.0622e-04,  ..., -5.1032e-04,
           -3.3032e-04, -1.5532e-04],
          ...,
          [-3.8437e-04, -6.4759e-04, -6.8077e-04,  ..., -2.8786e-04,
           -4.3881e-04, -1.3633e-04],
          [-2.7306e-04, -3.3358e-04, -5.4119e-04,  ..., -3.2097e-04,
           -1.1874e-04, -5.5010e-04],
          [ 2.7541e-05, -2.3222e-04, -4.3750e-04,  ..., -2.1435e-04,
           -1.6358e-05, -2.9488e-04]],

         [[-3.5753e-04,  3.6441e-04, -2.3151e-04,  ...,  5.2989e-04,
            2.3138e-05, -5.2995e-04],
          [ 1.3873e-04,  1.9303e-04,  3.0827e-04,  ..., -5.7088e-04,
           -8.4425e-04,  1.3354e-04],
          [-1.0346e-04,  5.7335e-04,  1.3125e-04,  ..., -2.4151e-04,
           -2.6299e-04, -1.4248e-04],
          ...,
     